In [1]:
import csv
import random
from sklearn import metrics
import tensorflow as tf
from sklearn.metrics import f1_score
import numpy as np

import pickle

# custom functions
%load_ext autoreload
%autoreload 2
import util_helpers
import cnn_model
#import cnn_att
#import lstm_model
import lstm_att_model

ImportError: No module named tensorflow

## Reading File

In [115]:
with open('../labeling_tweets/Data/batch1_to_12.csv', 'rb') as csvfile:
    twitter_flu_reader = csv.reader(csvfile)
    twitter_flu_list = list(twitter_flu_reader)    
random.shuffle(twitter_flu_list)

In [116]:
len(twitter_flu_list)

6975

In [117]:
tweets= [row[1] for row in twitter_flu_list]
labels = [row[2] for row in twitter_flu_list]

## Pre Processing Tweets

In [118]:
tweets_text = util_helpers.pre_process_tweets (tweets)

In [119]:
MAX_VOCAB = None # to limit original number of words (None if no limit)
data_vectorized, dictionary, MAX_VOCAB, tokenizer = util_helpers.vectorize_tweets(tweets_text, MAX_VOCAB, verbose = True)

Vocabulary size: 6717
Average note length: 15.9840860215
Max note length: 36


In [120]:
#save tokenizer
with open("keras_tokenizer.pickle", "wb") as f:
    pickle.dump(tokenizer, f)

In [121]:
MAX_SEQ_LENGTH = 35
tweets_data, MAX_SEQ_LENGTH = util_helpers.pad_tweets(data_vectorized, MAX_SEQ_LENGTH)

print("Final Vocabulary: %s" % MAX_VOCAB)
print("Final Max Sequence Length: %s" % MAX_SEQ_LENGTH)


Final Vocabulary: 6717
Final Max Sequence Length: 35


## Pre Processing Labels

In [123]:
v_labels = util_helpers.vectorize_label(labels)

## Splitting files

In [124]:
#train_tweets, dev_tweets, test_tweets = util_helpers.split_data (tweets_data)
#train_labels, dev_labels, test_labels = util_helpers.split_data (v_labels)
train_tweets, dev_tweets= util_helpers.split_data2 (tweets_data,train_frac = 0.85)
train_labels, dev_labels = util_helpers.split_data2 (v_labels,train_frac = 0.85)
print 'Training set samples:', len (train_tweets)
print 'Dev set samples:', len (dev_tweets)
#print 'Test set samples:', len (test_tweets)

Training set samples: 5928
Dev set samples: 1047


In [125]:
# class weights
from sklearn.utils import class_weight
labels_list =np.unique(train_labels)
class_weight = class_weight.compute_class_weight('balanced', labels_list, train_labels)
class_weight_dict = {labels_list[0]: class_weight[0], labels_list[1]: class_weight[1] }
class_weight_dict

{0: 0.56317689530685922, 1: 4.4571428571428573}

In [126]:
tp_dev  = sum (dev_labels)
tp_dev

122

## LSTM with attention Model

In [149]:
reload(lstm_att_model)

NUM_CLASSES = 1
EMBEDDING_DIM = 100
#### build model
model = lstm_att_model.build_lstm_att_model(input_seq_length=MAX_SEQ_LENGTH, max_vocab = MAX_VOCAB,
                             external_embeddings = False,
                             embedding_dim=EMBEDDING_DIM, embedding_matrix= None,
                             training_dropout_keep_prob=0.5,
                             num_classes=NUM_CLASSES )





____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_20 (InputLayer)            (None, 35)            0                                            
____________________________________________________________________________________________________
Embedding (Embedding)            (None, 35, 100)       671800      input_20[0][0]                   
____________________________________________________________________________________________________
lstm_8 (LSTM)                    (None, 35, 35)        19040       Embedding[0][0]                  
____________________________________________________________________________________________________
dropout_34 (Dropout)             (None, 35, 35)        0           lstm_8[0][0]                     
___________________________________________________________________________________________

In [150]:
# Train the model
model.fit(train_tweets, train_labels, batch_size=50, epochs=5, validation_data=(dev_tweets, dev_labels), 
          class_weight=class_weight_dict, verbose=2)

Train on 5928 samples, validate on 1047 samples
Epoch 1/5
9s - loss: 0.6670 - acc: 0.6090 - val_loss: 0.4855 - val_acc: 0.8548
Epoch 2/5
8s - loss: 0.4877 - acc: 0.8171 - val_loss: 0.5240 - val_acc: 0.7813
Epoch 3/5
9s - loss: 0.4041 - acc: 0.8482 - val_loss: 0.5005 - val_acc: 0.7937
Epoch 4/5
9s - loss: 0.3468 - acc: 0.8785 - val_loss: 0.4888 - val_acc: 0.8071
Epoch 5/5
8s - loss: 0.2989 - acc: 0.9027 - val_loss: 0.4170 - val_acc: 0.8510


In [151]:
pred_train = model.predict(train_tweets, batch_size=50)
pred_dev = model.predict(dev_tweets, batch_size=50)


In [152]:
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(dev_labels,pred_dev, pos_label = 1)
roc_auc = metrics.auc(fpr, tpr)
roc_auc

0.89990252547629601

In [153]:
util_helpers.show_scores( dev_labels,pred_dev)

F1 scores
threshold   |  tn   |  fp  |   fn   |  tp   | f1 score                  | recall  |precision|
-----------------------------------------------------------------------------------------------
0.050:        124     801       0     122    ( 0.236 ,  0.233 ) = 0.235   1.000     0.132  
-----------------------------------------------------------------------------------------------
0.100:        549     376       7     115    ( 0.741 ,  0.375 ) = 0.634   0.943     0.234  
-----------------------------------------------------------------------------------------------
0.150:        618     307      13     109    ( 0.794 ,  0.405 ) = 0.694   0.893     0.262  
-----------------------------------------------------------------------------------------------
0.200:        671     254      14     108    ( 0.834 ,  0.446 ) = 0.744   0.885     0.298  
-----------------------------------------------------------------------------------------------
0.250:        703     222      17     105    ( 0

In [154]:
model.save('lstm_model.h5')

## CNN

In [134]:
reload(cnn_model)

NUM_CLASSES = 1
EMBEDDING_DIM = 100
#### build model
cnn_tmodel = cnn_model.build_cnn_model (input_seq_length=MAX_SEQ_LENGTH, max_vocab = MAX_VOCAB,
                            external_embeddings = False,
                             embedding_dim=EMBEDDING_DIM, embedding_matrix= None,        
                                   num_filters = 100, filter_sizes=[2,3,4,5,6], 
                                   training_dropout_keep_prob=0.5,
                           num_classes=NUM_CLASSES )



____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_17 (InputLayer)            (None, 35)            0                                            
____________________________________________________________________________________________________
embedding (Embedding)            (None, 35, 100)       671800      input_17[0][0]                   
____________________________________________________________________________________________________
conv1d_56 (Conv1D)               (None, 34, 100)       20100       embedding[0][0]                  
____________________________________________________________________________________________________
conv1d_57 (Conv1D)               (None, 33, 100)       30100       embedding[0][0]                  
___________________________________________________________________________________________

In [135]:
cnn_tmodel.fit(train_tweets, train_labels, batch_size=50, epochs=5, validation_data=(dev_tweets, dev_labels), 
          class_weight=class_weight_dict, verbose=2)

Train on 5928 samples, validate on 1047 samples
Epoch 1/5
14s - loss: 0.5869 - acc: 0.6581 - val_loss: 0.4499 - val_acc: 0.8042
Epoch 2/5
16s - loss: 0.3973 - acc: 0.8359 - val_loss: 0.3537 - val_acc: 0.8481
Epoch 3/5
13s - loss: 0.2881 - acc: 0.8785 - val_loss: 0.4467 - val_acc: 0.8080
Epoch 4/5
12s - loss: 0.2199 - acc: 0.9163 - val_loss: 0.4723 - val_acc: 0.8185
Epoch 5/5
13s - loss: 0.1660 - acc: 0.9406 - val_loss: 0.4072 - val_acc: 0.8558


In [136]:
pred_train = cnn_tmodel.predict(train_tweets, batch_size=50)
pred_dev = cnn_tmodel.predict(dev_tweets, batch_size=50)
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(dev_labels,pred_dev, pos_label = 1)
roc_auc = metrics.auc(fpr, tpr)
roc_auc

0.88251661497563139

In [137]:
util_helpers.show_scores( dev_labels,pred_dev)

F1 scores
threshold   |  tn   |  fp  |   fn   |  tp   | f1 score                  | recall  |precision|
-----------------------------------------------------------------------------------------------
0.050:        570     355      11     111    ( 0.757 ,  0.378 ) = 0.650   0.910     0.238  
-----------------------------------------------------------------------------------------------
0.100:        631     294      14     108    ( 0.804 ,  0.412 ) = 0.706   0.885     0.269  
-----------------------------------------------------------------------------------------------
0.150:        669     256      18     104    ( 0.830 ,  0.432 ) = 0.738   0.852     0.289  
-----------------------------------------------------------------------------------------------
0.200:        705     220      22     100    ( 0.854 ,  0.452 ) = 0.769   0.820     0.312  
-----------------------------------------------------------------------------------------------
0.250:        736     189      24      98    ( 0

In [138]:
model.save('cnn_model.h5')